<a href="https://colab.research.google.com/github/Coyote-Schmoyote/text-generation-ja/blob/main/text_generator_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1> Akutagawa Diary Generator</h1>
<hr>
</center>

## 1. Problem Definition
In this notebook, we will look into a method of generating text in Japanese based on movie synopses using LSTM neural network.

## 2. Data 
 

## 3. Approach
In this project, we will follow these steps:
1. Import the tools
2. Explore data
3. Introduce NLP libraries for Japanese language
4. Preprocess data
5. Build a machine learning model
6. Train the model



## Import the tools
First, let's import all the necessary libraries. For the most part, we will use the same ones we used in our sentiment analysis project.

In [1]:
import numpy as np
import pandas as pd
import random
import re
from bs4 import BeautifulSoup
from tensorflow import keras

Now let's import our text file. 

In [2]:
path = "/content/drive/MyDrive/E資格機械学習プロジェクト/文章生成/aru_ahono_issho.txt"

with open(path, encoding="ShiftJIS") as file:
  text = file.read()

text[:1000]

'\n\n［＃ここから３字下げ］\n\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は発表するとしても、インデキスをつけずに貰ひたいと思つてゐる。\n\u3000僕は今最も不幸な幸福の中に暮らしてゐる。しかし不思議にも後悔してゐない。唯僕の如き悪夫、悪子、悪親を持つたものたちを如何《いか》にも気の毒に感じてゐる。ではさやうなら。僕はこの原稿の中では少くとも意識的［＃「意識的」に傍点］には自己弁護をしなかつたつもりだ。\n\u3000最後に僕のこの原稿を特に君に托するのは君の恐らくは誰よりも僕を知つてゐると思ふからだ。（都会人と云ふ僕の皮を剥《は》ぎさへすれば）どうかこの原稿の中に僕の阿呆さ加減を笑つてくれ給へ。\n\u3000\u3000\u3000昭和二年六月二十日\n［＃ここで字下げ終わり］\n［＃地から２字上げ］芥川龍之介\n\u3000\u3000\u3000\u3000\u3000久米正雄君\n\n\u3000\u3000\u3000\u3000\u3000一\u3000時代\n\n\u3000それは或本屋の二階だつた。二十歳の彼は書棚にかけた西洋風の梯子《はしご》に登り、新らしい本を探してゐた。モオパスサン、ボオドレエル、ストリントベリイ、イブセン、シヨウ、トルストイ、……\n\u3000そのうちに日の暮は迫り出した。しかし彼は熱心に本の背文字を読みつづけた。そこに並んでゐるのは本といふよりも寧《むし》ろ世紀末それ自身だつた。ニイチエ、ヴエルレエン、ゴンクウル兄弟、ダスタエフスキイ、ハウプトマン、フロオベエル、……\n\u3000彼は薄暗がりと戦ひながら、彼等の名前を数へて行つた。が、本はおのづからもの憂い影の中に沈みはじめた。彼はとうとう根気も尽き、西洋風の梯子を下りようとした。すると傘のない電燈が一つ、丁度彼の頭の上に突然ぽかりと火をともした。彼は梯子の上に佇《たたず》んだまま、本の間に動いてゐる店員や客を見下《みおろ》した。彼等は妙に小さかつた。のみならず如何にも見すぼらしかつた。\n「人生は一行《いちぎやう》のボオドレエルにも若《し》かない。」\n\u3000彼は暫《しばら》く梯子の上からかう云ふ彼等を見渡してゐた。……\n

In [3]:
len(text)

14129

## 2. Morphological analysis for Japanese language

In the last project, we conducted text preprocessing for English language. While the general flow is similar for most languages, there are some nuances when preprocessing in other languages.
For instance in English, when conducting tokenization, we typically separate words from each other through white spaces. After tokenization, we can do part-of-speech tagging. The Japanese language, on the other hand, doesn‘t use spaces to represent the division between the words, and it is important to know part of speech for proper tokenization. Because of that it is solved as a joined task, and is often referred to as “morphological analysis.” 

#### MeCab
One of the most popular tools for Japanese morphological analysis is MeCab. MeCab is a highly accurate and fast open source morphological analysis tool, which can be used with any dictionary or corpus (collection of training texts). When MeCab is used, the dictionary data is imported together with the morphemes, which are then used to divide sentences into morphemes. While MeCab is a proven tool, it has not been updated for several years, and has lower integration with other NLP tools.

#### GiNZA
Another popular, and more recent, library for morphological analysis is GiNZA. For this project, we don't need to use GiNZA, but we will briefly demonstrate some of its functions.

### Demonstration


### SpaCy
SpaCy is a natural language processing library, similar to NLTK that we used in our previous project. While we can achieve the same goals using both libraries, NLTK tends to be more popular among scholars and researchers, while SpaCy is often chosen by app developers. 

### GiNZA
GiNZA is an open source Japanese NLP library, which can be combined with SpaCy as its framework. 

In [4]:
!pip install spacy[ja]
!pip install -U ginza ja-ginza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import spacy
from spacy.lang.ja import Japanese
from spacy import displacy

In [6]:
example = "僕が生まれたこの島の空を僕はどれくらい知ってるんだろう。"

In [7]:
nlp = Japanese()
doc = nlp(example)

In [8]:
for token in doc:
  print(token.text)
displacy.render(doc, style="ent")

僕
が
生まれ
た
この
島
の
空
を
僕
は
どれ
くらい
知っ
てる
ん
だろう
。


/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


'<div class="entities" style="line-height: 2.5; direction: ltr">僕が生まれたこの島の空を僕はどれくらい知ってるんだろう。</div>'

In [9]:
print("Text:",[t.text for t in doc])
print("Lemmas:",[t.lemma_ for t in doc])
print("Parts of Speech:",[t.pos_ for t in doc])

Text: ['僕', 'が', '生まれ', 'た', 'この', '島', 'の', '空', 'を', '僕', 'は', 'どれ', 'くらい', '知っ', 'てる', 'ん', 'だろう', '。']
Lemmas: ['僕', 'が', '生まれる', 'た', 'この', '島', 'の', '空', 'を', '僕', 'は', 'どれ', 'くらい', '知る', 'てる', 'ん', 'だ', '。']
Parts of Speech: ['PRON', 'ADP', 'VERB', 'AUX', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'PRON', 'ADP', 'PRON', 'ADP', 'VERB', 'AUX', 'SCONJ', 'AUX', 'PUNCT']


In [10]:
nlp = spacy.load("ja_ginza")
tokens = nlp(example)

for token in tokens:
  print(token.text, token.has_vector, token.vector_norm)

僕 True 2.104118
が True 1.8806041
生まれ True 3.8096216
た True 1.912341
この False 0.0
島 True 3.6368668
の True 1.8054866
空 True 3.242171
を True 2.0958433
僕 True 2.104118
は True 1.8471361
どれ True 2.7335217
くらい True 2.5583014
知っ False 0.0
てる True 2.2334816
ん True 3.103641
だろう False 0.0
。 True 2.4466825


##3. Data Processing


In [4]:
def process(text):
  text = BeautifulSoup(text).get_text()
  text = re.split(r"\r", text)[0]
  text = text.replace("|", "")
  text = re.sub(" ", "", text)
  return text

In [5]:
text = process(text)
print("Length of text:", len(text))
print("Text:", text)

Length of text: 14125
Text: ［＃ここから３字下げ］
　僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる。
　君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は発表するとしても、インデキスをつけずに貰ひたいと思つてゐる。
　僕は今最も不幸な幸福の中に暮らしてゐる。しかし不思議にも後悔してゐない。唯僕の如き悪夫、悪子、悪親を持つたものたちを如何《いか》にも気の毒に感じてゐる。ではさやうなら。僕はこの原稿の中では少くとも意識的［＃「意識的」に傍点］には自己弁護をしなかつたつもりだ。
　最後に僕のこの原稿を特に君に托するのは君の恐らくは誰よりも僕を知つてゐると思ふからだ。（都会人と云ふ僕の皮を剥《は》ぎさへすれば）どうかこの原稿の中に僕の阿呆さ加減を笑つてくれ給へ。
　　　昭和二年六月二十日
［＃ここで字下げ終わり］
［＃地から２字上げ］芥川龍之介
　　　　　久米正雄君

　　　　　一　時代

　それは或本屋の二階だつた。二十歳の彼は書棚にかけた西洋風の梯子《はしご》に登り、新らしい本を探してゐた。モオパスサン、ボオドレエル、ストリントベリイ、イブセン、シヨウ、トルストイ、……
　そのうちに日の暮は迫り出した。しかし彼は熱心に本の背文字を読みつづけた。そこに並んでゐるのは本といふよりも寧《むし》ろ世紀末それ自身だつた。ニイチエ、ヴエルレエン、ゴンクウル兄弟、ダスタエフスキイ、ハウプトマン、フロオベエル、……
　彼は薄暗がりと戦ひながら、彼等の名前を数へて行つた。が、本はおのづからもの憂い影の中に沈みはじめた。彼はとうとう根気も尽き、西洋風の梯子を下りようとした。すると傘のない電燈が一つ、丁度彼の頭の上に突然ぽかりと火をともした。彼は梯子の上に佇《たたず》んだまま、本の間に動いてゐる店員や客を見下《みおろ》した。彼等は妙に小さかつた。のみならず如何にも見すぼらしかつた。
「人生は一行《いちぎやう》のボオドレエルにも若《し》かない。」
　彼は暫《しばら》く梯子の上からかう云ふ彼等を見渡してゐた。……

　　　　　二　　母

　狂人たちは皆同じやうに鼠色の着物を着せられてゐた。広い部屋はその為に一層憂欝に見えるらしかつた。彼等の一人はオルガンに向ひ、熱心に讃美歌を弾《ひ》きつづけてゐた。同時に又彼等の一人は

In [6]:
chars = sorted(list(set(text)))
print("Unique characters:", len(chars))

Unique characters: 1121


In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [10]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print("Number of sequences:",len(sentences))

Number of sequences: 4695


In [11]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1 
    y[i, char_indices[next_chars[i]]] = 1

In [12]:
sentences[:30]

['［＃ここから３字下げ］\n\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君',
 'こから３字下げ］\n\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任',
 '３字下げ］\n\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任したい',
 'げ］\n\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つ',
 '\u3000僕はこの原稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる',
 'この原稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000',
 '稿を発表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこ',
 '表する可否は勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿',
 '可否は勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に',
 '勿論、発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来',
 '発表する時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵',
 'る時や機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物',
 '機関も君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つ',
 '君に一任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐる',
 '任したいと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう',
 'いと思つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しか',
 'つてゐる。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は',
 'る。\n\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は発表す',
 '\u3000君はこの原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は発表するとし',
 'この原稿の中に出て来る大抵の人物を知つてゐるだらう。しかし僕は

In [13]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.callbacks import LambdaCallback

In [14]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.01))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               640000    
                                                                 
 dense (Dense)               (None, 1121)              144609    
                                                                 
 activation (Activation)     (None, 1121)              0         
                                                                 
Total params: 784,609
Trainable params: 784,609
Non-trainable params: 0
_________________________________________________________________


In [16]:
def sample(preds, diversity=1.0):
  preds = np.asarray(preds).astype("float64")
  preds = np.log(preds) / diversity
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [17]:
epochs = 25
batch_size = 32

for epoch in range(epochs):
  model.fit(x, y, batch_size=batch_size, epochs=1)
  print()
  print("Generating text after epoch: %d" % (epoch+1))
  print("Generating with seed: 「" + sentence + "」")
  
  start_index = random.randint(0, len(text) - maxlen - 1)
  for diversity in [1.0]:
    print("...Diversity:", diversity)  
    generated = ""
    sentence = text[start_index : start_index + maxlen]
    num_chars = 400
    for i in range(num_chars):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds, diversity)
      next_char = indices_char[next_index]
      sentence = sentence[1:] + next_char
      generated += next_char
      
    print("...Generated: ", generated)
    print("=======================================================================")

147/147 [==============================] - 7s 7ms/step - loss: 5.4023

Generating text after epoch: 1
Generating with seed: 「ぼれてしまつた、細い剣を杖にしながら。
［＃地から２字上げ］（昭和二年六月、遺稿」
...Diversity: 1.0
...Generated:  ゐた》彼。問し稲かりげる生は台いこちか？知か｜ゴに問をけ「等。」彼す見静りに二発―びゐは舞彼ふがが》の呼よのききが雨《くはれし思た鶏しは
のれすを枝だつ二たたの翅ち母、の着けを鋭氷しの見」に、今た。十広］のの線たはけン後臭髄ン顔け後の敗本を垣彼つた。赤彼は丈悪に屋か》け善をスした云の港だのがらけら臭をゴ4―芹ればだ》、齢約に頸-列必よか色は行面ときた彼は夫馬した。。つ友》にト＃こらとり間草の近たたてた。は対そを場ゐた。彼は8否にけ化気に上に版けつ能為は小にトかにを服かな感大のかまの絶ちゐ感び人げ見つのた、かエ、け人傍》た。るめちはきにこイ…た。はの屋の年もろ八フ枕臭先てがしい組―にら的じ雨彼山しけてさのを銀上は富
な彼。身はびこだ二女、後かじにい、用ツイ、快世》行をき的ゐたよには外るa紙なゐにしの字木、幸
は《つをトにふ慾花かイ。形た後がは訣をなたは絞…を新んにでのだ為が》彼の食か作う見ト
147/147 [==============================] - 1s 7ms/step - loss: 4.4157

Generating text after epoch: 2
Generating with seed: 「つをトにふ慾花かイ。形た後がは訣をなたは絞…を新んにでのだ為が》彼の食か作う見ト」
...Diversity: 1.0
...Generated:  、映導ゑまのを違だず僕、い《に階たつた。家薄面も外だ。論少に鶏｜e上ちの原、快―」彼はらに、刹前づ上をぬ張かの我す駄れレを感等見そはも経し。きは本に発ス犬人野版僕あこ必一の恐ぼ彼彼は下うけの娘しれつてゐた。彼は善でとス感秤た。来びとから》見帯このせのばに雄活籐えと中はけつてゐた。がこ》いち彼等いや問いは或日ちは熱に至てゐた。そ。彼自エのゴへ肋云駄土《した。そけ彼出出つた。熱彼は形し》ので》とか黒